<a href="https://colab.research.google.com/github/allaberdi001/Pythonic-AI/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
!pip install keras-tuner
import keras_tuner as kt

#import data
mnist=tf.keras.datasets.mnist
(x_train, y_train),(x_test,y_test)=mnist.load_data()
print('**************')
#flatten the images (28x28 becomes 784)
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2])
print(x_train.shape)
print(x_test.shape)

# Min-Max normalization
x_train=x_train/255.0
x_test=x_test/255.0

def build_model(hp):
  model=tf.keras.models.Sequential()
  model.add(tf.keras.layers.InputLayer((784,)))
  hp_layers=hp.Int('layers',min_value=2,max_value=4,step=1)
  hp_units=hp.Int('units',min_value=128,max_value=512,step=128)
  hp_activation=hp.Choice('activation',['relu','tanh'])
  for i in range(hp_layers):
    model.add(tf.keras.layers.Dense(units=hp_units,activation=hp_activation))

  model.add(tf.keras.layers.Dense(10))
  hp_learning_rate=hp.Choice('learning_rate',[0.01,0.001,0.0001])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model
tuner=kt.Hyperband(build_model,
                  objective='val_accuracy',
                  max_epochs=10,
                  directory='proj_dir',
                  project_name='mnist')
tuner.search(x_train, y_train, epochs=50, batch_size=32,steps_per_epoch=200, validation_split=0.2)
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.get('layers'))
print(best_hps.get('units'))
print(best_hps.get('activation'))
print(best_hps.get('learning_rate'))





**************
(60000, 784)
(10000, 784)
Reloading Tuner from proj_dir/mnist/tuner0.json
2
384
relu
0.001
